# Mandelbrot set in Tensorflow: unrolling iteration steps as dataflow graph

In [1]:
# imports for simulation
import tensorflow as tf
import numpy as np

# imports for visualization
import PIL.Image
from io import BytesIO
from IPython.display import clear_output, Image, display

# imports for interaction 
from ipywidgets import interact, FloatSlider
import time

sess = tf.InteractiveSession()

In [2]:
class Timer:
    def __init__(self):
        self.perf_start = time.perf_counter()
        self.proc_start = time.process_time()
    def report(self):
        return str(round((time.perf_counter()-self.perf_start),3))
    def reset(self):
        result = self.report()
        self.perf_start = time.perf_counter()
        self.proc_start = time.process_time()
        return result

In [ ]:
def DisplayFractal(a, fmt='jpeg'):
  """given a 2D array of escape-time iteration counts, 
     render as an image"""
  ashape = a.shape
  a_cyclic = (6.28*a/20.0).reshape(list(a.shape)+[1])
  img = np.concatenate([10+20*np.cos(a_cyclic),
                        30+50*np.sin(a_cyclic),
                        155-80*np.cos(a_cyclic)], 2)
  img[a==a.max()] = 0     # if reach max counts then not diverged, so setting  to 0
  a = img
  print(img.size)
  a = np.uint8(np.clip(a, 0, 255))
  f = BytesIO()
  PIL.Image.fromarray(a).save(f, fmt)
  clear_output(wait = True)
  display(Image(data=f.getvalue()))
  print(" inshape:", ashape)
  print("  cshape:", a_cyclic.shape)
  print("outshape:", img.shape)

In [42]:
tim1 = Timer()
maxiter = 200
# numpy setup, result Z is 1D array of complex numbers
nY, nX = np.mgrid[-2.0:2.0:0.005, -2.0:2.0:0.005] 
nZ = nX+1j*nY
print(type(nZ))
print(nZ.shape)
print("numpy: " + tim1.reset())

sess.close()
tf.reset_default_graph()
sess = tf.InteractiveSession()
print("sess startup:", tim1.reset())

tzoom = tf.placeholder(np.complex64, shape=())
toffset = tf.placeholder(np.complex64, shape=())
C = tf.Variable(nZ.astype(np.complex64))
C = C/tzoom + toffset
Z = C
M = tf.Variable(tf.zeros_like(C, tf.float32))


for i in range(maxiter):
    Z = Z*Z + C
    not_diverged = (tf.abs(Z) <= 4)
    M = M + tf.cast(not_diverged, tf.float32)

# diverged = (tf.abs(Z) > 4)
# M = M * tf.cast(diverged, tf.float32)

print("graph setup:", tim1.reset())
tf.global_variables_initializer().run()
print("graph init:", tim1.reset())


<class 'numpy.ndarray'>
(800, 800)
numpy: 0.02
sess startup: 0.003
graph setup: 1.861
graph init: 0.106


In [43]:
def view_changed(zoomer, xoff, yoff): 
    vtime = Timer()
    zscaling = zoomer + 0j*zoomer
    zoffset = xoff + 1j*yoff
    result = M.eval(feed_dict = {tzoom: zscaling, toffset: zoffset})
    compute_time = vtime.reset()
    print(result.size)
    result2 = result.reshape(list(result.shape)+[1])
    print(result2.size)
    DisplayFractal(result)
    display_time = vtime.reset()
    print("compute time: " + compute_time)
    print("display time: " + display_time)
    
interact(view_changed, 
         zoomer = FloatSlider(min=1,max=50,step=0.1,value=1, continuous_update=True), 
         xoff = FloatSlider(min=-2, max=2, step=0.05, value=-0.5, continuous_update=True), 
         yoff = FloatSlider(min=-2, max=2, step=0.05, value=0.0, continuous_update=True))

<function __main__.view_changed>